<a href="https://colab.research.google.com/github/JordanDCunha/Hands-On-Machine-Learning-with-Scikit-Learn-and-PyTorch/blob/main/Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering Algorithms: k-Means and DBSCAN

Clustering is an **unsupervised learning** task where the goal is to group similar instances together **without labels**.

Unlike classification, clustering algorithms must discover structure in the data on their own.

Clustering is widely used in:
- Customer segmentation
- Data analysis
- Dimensionality reduction
- Feature engineering
- Anomaly detection
- Semi-supervised learning
- Image segmentation

There is no single definition of a “cluster.” Some algorithms look for:
- Centroid-based clusters
- Density-based clusters
- Hierarchical clusters

In this section, we focus on **k-means clustering**.


In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

# Generate synthetic dataset with 5 clusters
X, y = make_blobs(
    n_samples=1000,
    centers=5,
    cluster_std=0.6,
    random_state=42
)

plt.scatter(X[:, 0], X[:, 1], s=10)
plt.title("Unlabeled Dataset with 5 Blobs")
plt.show()


## k-Means Clustering

k-means is a **centroid-based clustering algorithm**.

It works by:
1. Randomly initializing `k` centroids
2. Assigning each instance to the nearest centroid
3. Updating centroids to be the mean of assigned points
4. Repeating until centroids stop moving

The value of `k` **must be specified in advance**.


In [ ]:
from sklearn.cluster import KMeans

k = 5
kmeans = KMeans(n_clusters=k, random_state=42)

y_pred = kmeans.fit_predict(X)


## Cluster Labels vs Class Labels

In clustering:
- Labels are **discovered**, not provided
- A label is simply a **cluster index**
- Labels have no inherent meaning like class labels

The labels assigned during training are stored in `labels_`.


In [ ]:
# First 10 predicted cluster labels
y_pred[:10]


In [ ]:
# Centroids found by k-means
kmeans.cluster_centers_


## Visualizing k-Means Clusters

k-means partitions space into **Voronoi regions**, where each point belongs to the nearest centroid.

It works best when clusters are:
- Spherical
- Similar in size
- Well separated


In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_pred, cmap="viridis", s=10)
plt.scatter(
    kmeans.cluster_centers_[:, 0],
    kmeans.cluster_centers_[:, 1],
    c="red",
    marker="x",
    s=100
)
plt.title("k-Means Clustering Result")
plt.show()


## Assigning New Instances to Clusters

Once trained, k-means can assign **new data points** to the nearest centroid.


In [ ]:
import numpy as np

X_new = np.array([
    [0, 2],
    [3, 2],
    [-3, 3],
    [-3, 2.5]
])

kmeans.predict(X_new)


## Hard vs Soft Clustering

- **Hard clustering**: each instance belongs to exactly one cluster
- **Soft clustering**: each instance has a score for every cluster

k-means supports soft clustering by measuring distances to centroids.


In [ ]:
# Distance from each instance to each centroid
kmeans.transform(X_new).round(2)


This transformation produces a **k-dimensional representation**, which can be used for:
- Nonlinear dimensionality reduction
- Feature engineering
- Feeding into another ML model


## The k-Means Algorithm (Summary)

The algorithm alternates between:
- Assigning instances to the closest centroid
- Updating centroids to the mean of assigned points

The algorithm is guaranteed to converge because the total squared distance to centroids always decreases.

However, it may converge to a **local optimum**, depending on centroid initialization.


## Strengths and Limitations of k-Means

### Strengths
- Very fast and scalable
- Simple and efficient
- Works well for spherical clusters

### Limitations
- Must choose `k` manually
- Sensitive to initialization
- Performs poorly with:
  - Different cluster sizes
  - Non-spherical shapes
  - Outliers


# Centroid Initialization Methods in k-Means

The k-means algorithm is sensitive to centroid initialization. Poor initialization can lead to suboptimal clustering.

There are several strategies to mitigate this issue.


In [ ]:
import numpy as np
from sklearn.cluster import KMeans


## Manual Initialization

If you already have a good idea of where centroids should be (for example, from a previous clustering run), you can manually specify them.


In [ ]:
good_init = np.array([
    [-3, 3],
    [-3, 2],
    [-3, 1],
    [-1, 2],
    [0, 2]
])

kmeans = KMeans(n_clusters=5, init=good_init, random_state=42)
kmeans.fit(X)


## Multiple Random Initializations (`n_init`)

Another approach is to run k-means multiple times with different random initializations and keep the best solution.

The `n_init` hyperparameter controls how many times the algorithm runs with different initial centroids.


In [ ]:
kmeans = KMeans(
    n_clusters=5,
    init="random",
    n_init=10,
    random_state=42
)

kmeans.fit(X)


## Inertia

To decide which initialization is best, k-means uses **inertia**.

**Inertia** is defined as the sum of squared distances between each instance and its closest centroid.


In [ ]:
kmeans.inertia_


The `score()` method returns the **negative inertia** (because higher scores are considered better in Scikit-Learn).


In [ ]:
kmeans.score(X)


## k-Means++

k-means++ is a smarter centroid initialization algorithm that spreads centroids far apart.

It greatly reduces the risk of poor clustering and usually converges faster.


### k-Means++ Algorithm

1. Choose one centroid randomly from the dataset.
2. Choose the next centroid with probability proportional to the squared distance from the closest existing centroid.
3. Repeat until k centroids are chosen.


In [ ]:
kmeans = KMeans(
    n_clusters=5,
    init="k-means++",
    random_state=42
)

kmeans.fit(X)


When using k-means++, `n_init` defaults to 1 because initialization quality is already high.


# Accelerated k-Means and Mini-Batch k-Means


## Elkan’s Accelerated k-Means

Elkan’s algorithm speeds up k-means by avoiding unnecessary distance computations using the triangle inequality.

It can speed up training on some datasets, but may slow it down on others.


In [ ]:
kmeans = KMeans(
    n_clusters=5,
    algorithm="elkan",
    random_state=42
)

kmeans.fit(X)


## Mini-Batch k-Means

Mini-batch k-means uses small random subsets of the data to update centroids.

This allows clustering of very large datasets that do not fit in memory.


In [ ]:
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(
    n_clusters=5,
    random_state=42
)

minibatch_kmeans.fit(X)


# Finding the Optimal Number of Clusters


## The Elbow Method

Inertia always decreases as k increases, so we look for an **elbow** in the inertia curve.


## The Elbow Method

Inertia always decreases as k increases, so we look for an **elbow** in the inertia curve.


In [ ]:
inertias = []
ks = range(1, 11)

for k in ks:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(ks, inertias, "bo-")
plt.xlabel("k")
plt.ylabel("Inertia")
plt.title("Elbow Method")
plt.show()


## Silhouette Score

The silhouette score measures how well instances fit within their cluster compared to other clusters.


In [ ]:
from sklearn.metrics import silhouette_score

kmeans = KMeans(n_clusters=5, random_state=42)
labels = kmeans.fit_predict(X)

silhouette_score(X, labels)


# Limits of k-Means

k-means struggles when:
- Clusters have different sizes
- Clusters have different densities
- Clusters are non-spherical


Always scale features before using k-means to improve performance.


# Image Segmentation Using k-Means


In [ ]:
import PIL.Image as Image

image = np.asarray(Image.open(filepath))
image.shape


In [ ]:
X_pixels = image.reshape(-1, 3)

kmeans = KMeans(n_clusters=8, random_state=42)
kmeans.fit(X_pixels)

segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)


In [ ]:
plt.imshow(segmented_img.astype(np.uint8))
plt.axis("off")
plt.show()


# Semi-Supervised Learning with k-Means


In [ ]:
from sklearn.datasets import load_digits

X_digits, y_digits = load_digits(return_X_y=True)
X_train, y_train = X_digits[:1400], y_digits[:1400]
X_test, y_test = X_digits[1400:], y_digits[1400:]


In [ ]:
from sklearn.linear_model import LogisticRegression

n_labeled = 50
log_reg = LogisticRegression(max_iter=10_000)
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

log_reg.score(X_test, y_test)


## Clustering the Training Set


In [ ]:
k = 50
kmeans = KMeans(n_clusters=k, random_state=42)
X_digits_dist = kmeans.fit_transform(X_train)

representative_idx = X_digits_dist.argmin(axis=0)
X_representative_digits = X_train[representative_idx]


(Representative labels are assumed to be manually provided.)


In [ ]:
y_representative_digits = np.array([
    8, 0, 1, 3, 6, 7, 5, 4, 2, 8,
    # ...
])


In [ ]:
log_reg = LogisticRegression(max_iter=10_000)
log_reg.fit(X_representative_digits, y_representative_digits)

log_reg.score(X_test, y_test)


# Label Propagation


In [ ]:
y_train_propagated = np.empty(len(X_train), dtype=np.int64)

for i in range(k):
    y_train_propagated[kmeans.labels_ == i] = y_representative_digits[i]


In [ ]:
log_reg = LogisticRegression(max_iter=10_000)
log_reg.fit(X_train, y_train_propagated)

log_reg.score(X_test, y_test)


# DBSCAN


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.05, random_state=42)

dbscan = DBSCAN(eps=0.2, min_samples=5)
dbscan.fit(X)


In [ ]:
dbscan.labels_


Instances labeled `-1` are considered anomalies.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(
    dbscan.components_,
    dbscan.labels_[dbscan.core_sample_indices_]
)


In [ ]:
X_new = np.array([
    [-0.5, 0],
    [0, 0.5],
    [1, -0.1],
    [2, 1]
])

knn.predict(X_new)


# Summary

- k-means is fast and scalable but sensitive to initialization and cluster shape
- k-means++ greatly improves reliability
- DBSCAN finds clusters of arbitrary shape and detects anomalies
- Different datasets require different clustering strategies


# Gaussian Mixture Models (GMMs)

A **Gaussian Mixture Model (GMM)** is a probabilistic model that assumes the data was generated from a mixture of several **Gaussian distributions** with unknown parameters.

Each Gaussian represents a **cluster**, and unlike k-means:
- Clusters can be **elliptical**, not just spherical
- Clusters can have **different sizes, orientations, and densities**
- Assignments are **soft** (probabilities), not hard

A GMM assumes:
- There are `k` clusters
- Each cluster `j` has:
  - weight \( \phi^{(j)} \)
  - mean \( \mu^{(j)} \)
  - covariance matrix \( \Sigma^{(j)} \)

Scikit-Learn implements this using the **Expectation-Maximization (EM)** algorithm.


In [ ]:
from sklearn.mixture import GaussianMixture

# Fit a Gaussian Mixture Model with 3 clusters
gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)


## Learned Parameters

After fitting, the model estimates:
- Cluster weights
- Means
- Covariance matrices


In [ ]:
gm.weights_


In [ ]:
gm.means_


In [ ]:
gm.covariances_


These correspond to:
- **weights** → relative size of each cluster  
- **means** → cluster centers  
- **covariances** → shape and orientation of clusters  


## Expectation–Maximization (EM) Algorithm

The **EM algorithm** alternates between two steps:

### Expectation step (E-step)
Estimate the probability that each instance belongs to each cluster  
(these probabilities are called **responsibilities**).

### Maximization step (M-step)
Update:
- cluster means
- covariance matrices
- cluster weights  

using the responsibilities.

Unlike k-means:
- All points influence all clusters
- Influence is weighted by probability

⚠️ EM can converge to poor local optima, so multiple initializations (`n_init`) are important.


In [ ]:
gm.converged_, gm.n_iter_


## Hard vs Soft Clustering

- **Hard clustering** assigns each instance to the most likely cluster
- **Soft clustering** returns probabilities for each cluster


In [ ]:
gm.predict(X)


In [ ]:
gm.predict_proba(X).round(3)


## Generative Models

GMMs are **generative models**, meaning we can sample new instances from the learned distributions.


In [ ]:
X_new, y_new = gm.sample(6)
X_new, y_new


## Density Estimation

The `score_samples()` method returns the **log probability density** at each point.

- Higher value → higher density
- These are **log-PDF values**, not probabilities


In [ ]:
gm.score_samples(X).round(2)


## Covariance Constraints

You can restrict cluster shapes using `covariance_type`:

| Type | Description |
|----|----|
| `"full"` | Any shape, size, orientation (default) |
| `"tied"` | All clusters share the same covariance |
| `"diag"` | Ellipses aligned with axes |
| `"spherical"` | Spheres with different radii |


In [ ]:
gm_spherical = GaussianMixture(
    n_components=3,
    covariance_type="spherical",
    n_init=10,
    random_state=42
)

gm_spherical.fit(X)


## Anomaly Detection with GMMs

Instances in **low-density regions** can be treated as anomalies.

A common approach:
1. Compute density scores
2. Choose a percentile threshold (e.g., lowest 2%)
3. Flag points below that threshold


In [ ]:
import numpy as np

densities = gm.score_samples(X)
threshold = np.percentile(densities, 2)

anomalies = X[densities < threshold]


## Choosing the Number of Clusters

Gaussian mixtures use **information criteria**:

- **AIC** → favors better fit
- **BIC** → favors simpler models

Lower values are better.


In [ ]:
gm.aic(X), gm.bic(X)


## Bayesian Gaussian Mixture Models

`BayesianGaussianMixture` can automatically eliminate unnecessary clusters by assigning them near-zero weight.


In [ ]:
from sklearn.mixture import BayesianGaussianMixture

bgm = BayesianGaussianMixture(
    n_components=10,
    n_init=10,
    max_iter=500,
    random_state=42
)

bgm.fit(X)
bgm.weights_.round(2)


## Limitations of Gaussian Mixtures

GMMs struggle when clusters:
- Are **non-ellipsoidal**
- Have complex shapes (e.g., two moons)

They may split one cluster into many ellipses.


## Other Anomaly and Novelty Detection Algorithms

Scikit-Learn also provides:

- **EllipticEnvelope** (robust Gaussian fit)
- **Isolation Forest** (random partitioning)
- **Local Outlier Factor (LOF)** (density comparison)
- **One-Class SVM** (novelty detection)
- **PCA reconstruction error**

Each method has different strengths depending on dataset size, dimensionality, and noise.
